In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shashwatwork/knee-osteoarthritis-dataset-with-severity")

print("Path to dataset files:", path)

Path to dataset files: /home/cpm6gh/.cache/kagglehub/datasets/shashwatwork/knee-osteoarthritis-dataset-with-severity/versions/1


In [9]:
import kagglehub
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image

In [10]:
path = "/home/cpm6gh/.cache/kagglehub/datasets/shashwatwork/knee-osteoarthritis-dataset-with-severity/versions/1"
train_dir = os.path.join(path, "train")
val_dir = os.path.join(path, "val")
test_dir = os.path.join(path, "test")

In [ ]:
import torch.nn.functional as F

def asymmetric_penalty_loss(outputs, targets):
    # outputs: raw logits of shape (batch_size, num_classes)
    # targets: ground truth labels of shape (batch_size)

    log_probs = F.log_softmax(outputs, dim=1)  # Convert logits to log probabilities
    probs = torch.exp(log_probs)  # shape: (batch_size, num_classes)
    
    batch_size, num_classes = probs.shape
    range_tensor = torch.arange(num_classes, device=targets.device).unsqueeze(0).expand(batch_size, -1)

    # Expand targets to match output shape
    targets_expanded = targets.unsqueeze(1).expand_as(probs)

    # Compute the penalty matrix
    penalty = torch.ones_like(probs)
    penalty[range_tensor < targets_expanded] = 1  # Underestimation penalty
    # Overestimation or correct guess has penalty = 1.0

    # Get the loss: Negative Log Likelihood weighted by penalty
    loss = -penalty * log_probs
    loss = loss.gather(1, targets.unsqueeze(1)).squeeze(1)
    
    return loss.mean()


In [14]:
class KneeOADataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for label in range(5):  # Labels: 0 to 4
            label_dir = os.path.join(root_dir, str(label))
            if os.path.exists(label_dir):
                for img_name in os.listdir(label_dir):
                    self.image_paths.append(os.path.join(label_dir, img_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [16]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [17]:
train_dataset = KneeOADataset(train_dir, transform=transform)
val_dataset = KneeOADataset(val_dir, transform=transform)
test_dataset = KneeOADataset(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)

/home/cpm6gh/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cpm6gh/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = asymmetric_penalty_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%")

# Save model
torch.save(model.state_dict(), "resnet_knee_oa.pth")

Epoch 1, Loss: 0.8960, Accuracy: 62.44%
Epoch 2, Loss: 0.8134, Accuracy: 66.25%
Epoch 3, Loss: 0.7610, Accuracy: 68.09%
Epoch 4, Loss: 0.7049, Accuracy: 70.08%
Epoch 5, Loss: 0.6284, Accuracy: 73.69%
Epoch 6, Loss: 0.5470, Accuracy: 76.93%
Epoch 7, Loss: 0.4836, Accuracy: 80.24%
Epoch 8, Loss: 0.4364, Accuracy: 82.21%
Epoch 9, Loss: 0.3283, Accuracy: 86.50%
Epoch 10, Loss: 0.2975, Accuracy: 88.18%


In [ ]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = asymmetric_penalty_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%")

# Save model
torch.save(model.state_dict(), "resnet_knee_oa.pth")

In [ ]:
val_loss, val_acc = evaluate_model(model, val_loader, asymmetric_penalty_loss, device)
print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")


In [ ]:
test_loss, test_acc = evaluate_model(model, test_loader, asymmetric_penalty_loss, device)
print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_acc:.2f}%")
